In [ ]:
!pip install -q pytorch

In [4]:
import torch
from torch import nn
from torch.nn import Embedding
from torch.distributions import Categorical

In [5]:
class PoincareModel(nn.Module):
    def __init__(self, embedding_dim, vocab_size, init_weights=1e-3, epsilon=1e-7):
        """
        Constructeur de la classe PoincareModel.

        Args:
            dim (int): Dimension des vecteurs d'embedding.
            size (int): Taille du dictionnaire (nombre d'entités distinctes).
            init_weights (float): Poids d'initialisation pour les embeddings.
            epsilon (float): Petit epsilon utilisé pour éviter les divisions par zéro.
        """
        super().__init__()
        

        # Crée une couche d'embedding avec des vecteurs d'embedding de dimension 'dim' pour 'size' entités.
        self.embedding = nn.Embedding(vocab_size, embedding_dim, sparse=False)

        # Initialise les poids d'embedding de manière aléatoire dans l'intervalle [-init_weights, init_weights].
        self.embedding.weight.data.uniform_(-init_weights, init_weights)

        # Stocke la valeur d'epsilon utilisée pour éviter les divisions par zéro dans la distance.
        self.epsilon = epsilon

        self.loss = nn.CrossEntropyLoss(ignore_index=-1)



    def dist(self, u, v):
        """
        Calcule la distance entre deux vecteurs d'embedding dans l'espace hyperbolique de Poincaré.

        Args:
            u (Tensor): Le premier vecteur d'embedding.
            v (Tensor): Le deuxième vecteur d'embedding.

        Returns:
            Tensor: La distance entre les deux vecteurs d'embedding.
        """
        # Calcule la distance de Poincaré entre u et v.
        sqdist = torch.sum((u - v) ** 2, dim=-1)
        squnorm = torch.sum(u ** 2, dim=-1)
        sqvnorm = torch.sum(v ** 2, dim=-1)
        x = 1 + 2 * sqdist / ((1 - squnorm) * (1 - sqvnorm)) + self.epsilon
        z = torch.sqrt(x ** 2 - 1)
        return torch.log(x + z)

    def forward(self, inputs):
        """
        Implémentation de la passe avant (forward) du modèle.

        Args:
            inputs (Tensor): Indices des entités en entrée.

        Returns:
            Tensor: Les distances entre les embeddings des entités.
        """
        # Obtient les embeddings des entités correspondant aux indices en entrée.
        e = self.embedding(inputs)
        
        # Sépare la première dimension (indice 0) pour obtenir le point de départ (s) et le reste (o).
        o = e.narrow(dim=1, start=1, length=e.size(1) - 1)
        s = e.narrow(dim=1, start=0, length=1).expand_as(o)
        
        # Calcule la distance entre les deux embeddings.
        return self.dist(s, o)



In [20]:
word_frequency = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
words = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j']
categorical_dist = Categorical(torch.tensor(word_frequency))
uniform_dist = Categorical(torch.ones(len(word_frequency)) / len(word_frequency))


tensor([[[-1.8215,  2.1824,  0.8829]]], grad_fn=<EmbeddingBackward0>)
